In [1]:
# 필요한 패키지와 라이브러리를 가져옴
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
# conda install -c conda-forge fbprophet 
from prophet import Prophet

mpl.rc('font',family='Malgun Gothic')   # 맑은 고딕으로 그래프한글설정

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [30]:
## file 불러옴
data = pd.read_csv('./data/Test.csv', encoding = 'cp949')
data.head()

,date,car_num,daily_distance,cum_distance,information_date,car_speed,RPM,brake_signal,car_location_GPS_X,car_location_GPS_Y,RPMS,ACC,FS,RPMSR,ACCR,SRA
0,20221201,2612,44,3093066,2.212190e+13,0,577,0,129.158634,35.236121,-29.5,0.0,0,0,0,0
1,20221201,2612,44,3093066,2.212190e+13,0,555,0,129.158634,35.236121,-1.5,0.0,0,0,0,0
2,20221201,2612,44,3093066,2.212190e+13,0,558,0,129.158634,35.236121,4.5,0.0,0,0,0,0
3,20221201,2612,44,3093066,2.212190e+13,0,546,0,129.158634,35.236121,-6.5,0.0,0,0,0,0
4,20221201,2612,44,3093066,2.212190e+13,0,561,0,129.158634,35.236121,2.0,0.0,0,0,0,0


In [11]:
type(data)

pandas.core.frame.DataFrame

In [42]:
#필요한 열 추출
data = data[['date','car_num','daily_distance']]
data

,date,car_num,daily_distance
0,20221201,2612,44
1,20221201,2612,44
2,20221201,2612,44
3,20221201,2612,44
4,20221201,2612,44
...,...,...,...
168083,20221214,9357,88
168084,20221214,9357,88
168085,20221214,9357,88
168086,20221214,9357,88


In [51]:
# 날짜별, 차량별로 일일주행거리 최대값만 추출
data1 = data.loc[data.groupby(['date','car_num'])['daily_distance'].idxmax()]
data1

,date,car_num,daily_distance
1288,20221201,2612,88
33571,20221201,3897,88
48697,20221201,4699,44
65235,20221201,4985,44
81829,20221201,5221,135
...,...,...,...
113548,20221214,5634,220
131547,20221214,8993,44
149314,20221214,9007,176
168074,20221214,9357,88


In [52]:
# 컬럼 리스트 생성
date = data1.iloc[:,0].to_list()
car_num = data1.iloc[:,1].to_list()
daily_distance = data1.iloc[:,2].to_list()

In [56]:
list_sqlData = []
# sql문 생성
for i in range(len(data1)) :
    a = date[i], car_num[i], daily_distance[i]
    list_sqlData.append(a)
list_sqlData

list

In [57]:
## list를 MySQL DB에 넣음
import pymysql
conn = pymysql.connect(host='database-1.cd1ds11df3el.ap-northeast-2.rds.amazonaws.com',
                                     database='db1',
                                     user='admin',
                                     password='tigertiger')
try:
    with conn.cursor() as cursor:
        sql = 'INSERT INTO oil (date, car_num, daily_distance) VALUES (%s, %s, %s)'
        cursor.executemany(sql, list_sqlData)
    conn.commit()
finally:
    conn.close()